<a id="top"></a>
# Robotic Instrument Segmentation Demo

## Check for latest version
<br><div class=danger><b>Important: Before proceeding, please run the following cell to ensure that you are running the most recent version of this sample.</b></div>

In [ ]:
# Show the current status of this and all documents with ability to update
from qarpo.catalog import DemoCatalog
import os
status = DemoCatalog(os.getcwd(), "Demo").ShowRepositoryControls()

## Introduction

This sample application demonstrates how a smart video IoT solution may be created using Intel® hardware and software tools to perform robotic instrument segmentation.  This solution performs semantic segmentation to identify the segments of robotic instruments within a video frame.  The identified robotic instruments segments are then highlighted in the output with each segment appearing in a different color.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Video and image input is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of the resulting segmentation in the output
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)


### Application background
![Robotic Instrument Challenge](./figures/segmentation.gif)

The code in this sample refers to the winning solution by Alexey Shvets, Alexander Rakhlin, Alexandr A. Kalinin, and Vladimir Iglovikov in the [MICCAI 2017 Robotic Instrument Segmentation Challenge](https://endovissub2017-roboticinstrumentsegmentation.grand-challenge.org/). This Jupyter* Notebook has been modified from the original found on [GitHub](https://github.com/ternaus/robot-surgery-segmentation/blob/master/Demo.ipynb) which is made available with an [MIT license](https://github.com/ternaus/robot-surgery-segmentation/blob/master/LICENSE). The data files necessary to run this notebook are included in `/data/robotic-surgery-segmentation`.

![TernausNet](./figures/TernausNet.png)

## Robotic instrument segmentation application
The robotic instrument segmentation application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on an input video to locate robotic instruments within each frame.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Use the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) to create the inference model IR files needed to perform inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the robotic instrument segmentation application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the specified input video to identify segments of robotic instruments.  Once identified, each robotic instrument segment is colored to highlight it on the output video.

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and and Intel® Arria® 10 FPGA.  After inference on the input is completed, the output is stored in the appropriate `results/<JobNum>/` directory.  The results are then viewed within this Jupyter* Notebook using the `videoHTML` video playback utility.

The application and inference code for this sample is already implemented in the Python* file [`segmentation_parts.py`](./python/segmentation_parts.py) (and other helper `python/*.py` files).

The following sections will guide you through configuring and running the robotic instrument segmentation demo.

## Inference with PyTorch (Optional)
Run the following cell to perform inference with the PyTorch model on a compute node with an [Intel Core i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-) using the code in [pytorch_infer.py](python/pytorch_infer.py). 

<br><div class=note><i><b>Note: </b>The next cell uses `qsub` to submit a job to be run on a compute node because the login node running this Jupyter* Notebook does not have enough memory to run the task.</i></div>

In [ ]:
# Run the script
job_id_infer = !qsub python/run_py.sh -l nodes=1:i5-6500te -F "python/pytorch_infer.py" -N pytorch_core
if job_id_infer:
    print(job_id_infer[0])
    progressIndicator('results/', job_id_infer[0]+'.txt', "Inference", 0, 100)

<br><div class=danger><b>Wait!: </b>
Please wait for the progress bar and job to complete before proceeding to the next cell.
</div>

To view the original input image and the inference results, run the following cell

In [ ]:
# read, label, and display original input image
image = cv2.imread("generated/input.png")
plt.figure(1, figsize=(15, 15))
plt.subplot(121)
plt.axis('off')
plt.title("Input Image")
plt.imshow(image)

# read, lable, and display result image
mask = cv2.imread("generated/mask.png")
plt.subplot(122)
plt.axis('off')
plt.title("Segmentation")
plt.imshow(mask_overlay(image, mask));

## Converting PyTorch* to ONNX*

The ONNX* models need to be generated from the original PyTorch* models to be used with the Intel® Distribution of OpenVINO™ toolkit.  This is done by running [pytorch_to_onnx.py](python/pytorch_to_onnx.py). 

<br><div class=note><i><b>Note: </b>The next cell uses `qsub` to submit a job to be run on a compute node because the login node running this Jupyter* Notebook does not have enough memory to run the task.</i></div>

In [ ]:
# Run the script
job_id_onnx = !qsub python/run_py.sh -l nodes=1:i5-6500te -F "python/pytorch_to_onnx.py" -N pytorch_to_onnx
if job_id_onnx:
    print(job_id_onnx[0])
    progressIndicator('results/', job_id_onnx[0]+'.txt', "Conversion", 0, 100)

<br><div class=danger><b>Wait!: </b>
Please wait for the progress bar and job to complete before proceeding to the next cell.
</div>

The two ONNX* models that have now be generated are:
- **models/onnx/surgical_tools_parts.onnx** - Segmentation model used for identifying robotic instrument segments ("parts")
 - This model will be used to process video and measure performance in later steps 
- **models/onnx/surgical_tools.onnx** - Segmentation model used for identifying robotic instrument ("binary")
 - This model, along with the `surgical_tools_parts.onnx`, will be used in an extra inference run to show some visualization of the segmentation involved.

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.). For this sample, the two ONNX* models that will be used were just generated from PyTorch* in the previous step.



The `surgical_tools_parts.onnx` and `surgical_tools.onnx` files will need to be optimized using the Model Optimizer to create the necessary IR files.  We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the most common precision `FP16` to run on all devices that we target.

For this model, we will run the Model Optimizer using the format:
```bash
/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model <path_to_model> \
    --output_dir <path_to_output_directory> \
    --data_type <data_precision> \
    --move_to_preprocess \
    --scale_values [<scale_values>] \
    --mean_values [<channel_mean_values>]
```

The input arguments are as follows:
- **--input_model** : The model's input *.pb* file
- **--output_dir** : Output directory where to store the generated IR model files
- **--data_type** : The model's data type and precision (e.g. FP16, FP32, INT8, etc.)
- **--move_to_preprocess** : Move mean values to IR preprocess section
- **--scale_values** : Scaling (divide by, one per channel) value to apply to input values
- **--mean_values** : Mean values (one per channel) to be subtracted from input values before scaling

The input shape, scaling values, and mean values we will be using are specific to the model topology being used.  Using the appropriate values for the model that we will use, The complete command will look like the following:
```bash
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model <path_to_model> \
    --data_type <data_precision> \
    --output_dir models/<data_precision> \
    --move_to_preprocess \
    --scale_values [58.395,57.12,57.375] \
    --mean_values [123.75,116.28,103.58]
```
We will run the command once on each model with <*path_to_model*> set to `models/onnx/surgical_tools.onnx` and `models/onnx/surgical_tools_parts.onnx` and with <*data_precision*> set to `FP16`.  This may be changed (e.g. `FP32`, etc.) as needed to run inference on other different devices.

<br><div class=note><i><b>Note: </b>More information on how to use Model Optimizer to convert TensorFlow* models as well as specifying input shape and scaling parameters for common model topologies may be found at:[Converting a ONNX* Model](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_ONNX.html)
</i></div>


Run the following cell to use the Model Optimizer to create the model IR files:

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_onnx.py \
    --input_model "models/onnx/surgical_tools.onnx" \
    --output_dir models/ov/FP16/ \
    --data_type FP16 \
    --move_to_preprocess \
    --scale_values "[0.229, 0.224, 0.225]" \
    --mean_values "[0.485, 0.456, 0.406]"

!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_onnx.py \
    --input_model "models/onnx/surgical_tools_parts.onnx" \
    --output_dir models/ov/FP16/ \
    --data_type FP16 \
    --move_to_preprocess \
    --scale_values "[0.229, 0.224, 0.225]" \
    --mean_values "[0.485, 0.456, 0.406]"

!echo "\nAll IR files that were created:"
!find ./models -name "*.xml" -o -name "*.bin"

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning of a line is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The above command will also work in a terminal (with the '!' removed).</i></div>

As shown above from the output of the last `!find...` command, the required sets of IR model files (`*.xml` and `*.bin`) have been created.  

### Run the demo

Running the next cell will display an interactive user interface allowing you to submit jobs to run the demo on multiple edge compute nodes selected by hardware devices, view the output of each job, and compare performance results across jobs.

To run a job:
1. Select the desired option in the **Target node** list
2. Select the desired device in the **Target architecture** list
3. Click the **Submit** button

After the **Submit** button is clicked, a tab will appear for the new job with a label in the format "*status*: *JobID*".  Once the job status appears as "Done", the **Display output** button may be clicked to view the output for the job.

After one or more jobs are done, the performance results for each job may be plotted by clicking the **Plot results** button.  Results for each job will be potted as bar graphs for **inference time** and **frames per second**.  Lower values mean better performance for **inference time** and higher values mean better performance for **frames per second**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

Run the next cell to begin the interactive demo.

In [ ]:
# import necessary libraries
import json
import qarpo

# load job configurations for demo
with open('job_config.json') as json_file:  
    data = json.load(json_file)
    
# create and run the user job interface
job_interface = qarpo.Interface(data)
job_interface.displayUI()

## Next steps
- The complete [Robotic Instrument Segmentation Sample](../../developer_samples/surgery-segmentation-python/robotic-surgery-segmentation.ipynb) for this demo
- [Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) -  tutorials on using and creating Jupyter* Notebooks
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


## Citation
    @inproceedings{shvets2018automatic,
    title={Automatic Instrument Segmentation in Robot-Assisted Surgery using Deep Learning},
    author={Shvets, Alexey A and Rakhlin, Alexander and Kalinin, Alexandr A and Iglovikov, Vladimir I},
    booktitle={2018 17th IEEE International Conference on Machine Learning and Applications (ICMLA)},
    pages={624--628},
    year={2018}
    }
<br><div class=note><i><b>Note: </b>Citation appears exactly as it appears on source website</i></div>

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>